# Import Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
data = pd.read_excel(r'C:\Users\91709\Downloads\Work\UpWork\Pete\Data - Master - 210913.xlsx', sheet_name = 'Data')

# Personal Details

In [3]:
personal = data[['InternalID', 'Sex', 'DOB']] #Select Relevant Columns
personal = personal.drop_duplicates(keep = 'last') #Remove Duplicates

personal['DOB'] = personal['DOB'].astype(str) #Convert Date to String
def age(born): #Extract Age from DOB
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))
  
personal['Age'] = personal['DOB'].apply(age) #Add age column

personal['DOB'] = pd.to_datetime(personal['DOB'], format='%Y-%m-%d') #Convert date back to date format
personal['DOB'] = personal['DOB'].dt.strftime('%d-%m-%Y') #Convert it to d-m-Y format

# Weight

In [4]:
weight = data.dropna(subset = ['Appt Date']) #Drop rows which does't have Appt Date
weight = weight.drop_duplicates(subset=['InternalID', 'Appt Date'] , keep="last") #Drop duplicate rows

app_count = weight[['InternalID', 'Appt Date']] #Select relevant columns
app_count = app_count.groupby(['InternalID']).count() #Count no of visits by each patient
app_count = app_count.reset_index() #Get access to all columns
app_count.rename(columns = {'Appt Date': '# of Appts'}, inplace = True) #Rename columns accoring to template

personalnweightcount = personal.merge(app_count, how = 'outer', on = 'InternalID') #Merge it with personal details

temp2 = weight.groupby(['InternalID']) #Group data on patients to get the earliest and latest date
temp2 = temp2.agg(Minimum_Date=('Appt Date', np.min), Maximum_Date=('Appt Date', np.max))
temp2 = temp2.reset_index() #Get access to all columns
temp2['months_to_add'] = 90 #Get the date 3 months after the first visit
temp2['3 months'] = temp2['Minimum_Date'] + pd.TimedeltaIndex(temp2['months_to_add'], unit='D')

three_month_list = temp2['3 months'].to_list() #Put the dates created in 14, 15 inside a list
patient_list = temp2['InternalID'].to_list() #Put patient ID inside a list
three_month_exact_date = [] #Create an empty list
for (i,j) in zip(patient_list, three_month_list): #Get the date of the first visit after 90 days in a list
    temp_weight = weight[weight['InternalID'] == i]
    date_working = temp_weight[(temp_weight['Appt Date'] >= j)]
    date = date_working['Appt Date'].min()
    three_month_exact_date.append(date)
temp2['>=3 Month Appt Date'] = three_month_exact_date #Add the list to column

weight = weight[['InternalID', 'Appt Date', 'Weight (kg)']] #Select Relevant columns from the dataset created in 2

#Look up the weight at first, first appointment after 3 months and last appointment and rename the column as per template
output2 = temp2.merge(weight, how='left', left_on=["InternalID", "Minimum_Date"], \
                      right_on=["InternalID","Appt Date"])
output2.rename(columns = {'Minimum_Date': '1st Appt Date','Weight (kg)':'1st Weight (Kg)'}, inplace = True)
output2 = output2.merge(weight, how='left', left_on=["InternalID", ">=3 Month Appt Date"], \
                        right_on=["InternalID","Appt Date"])
output2.rename(columns = {'Weight (kg)':'>=3 Month Weight'}, inplace = True)
output2 = output2.merge(weight, how='left', left_on=["InternalID", "Maximum_Date"], \
                        right_on=["InternalID","Appt Date"])
output2.rename(columns = {'Maximum_Date':'Last Appt Date','Weight (kg)':'Last  Weight (Kg)'}, inplace = True)

output2= output2[['InternalID', '1st Appt Date', '1st Weight (Kg)',  '>=3 Month Appt Date','>=3 Month Weight', \
                  'Last Appt Date', 'Last  Weight (Kg)']] #Select Relevant columns

#Get absolute and % change between first & first appointment after 3 months and first and last appointment
output2.loc[(output2['1st Weight (Kg)'].isnull()) | (output2['>=3 Month Weight'].isnull()), \
            '1st - >=3 Month Appt Change In Weight (Kg)'] = None
output2.loc[(output2['1st Weight (Kg)'].notnull()) | (output2['>=3 Month Weight'].notnull()), \
            '1st - >=3 Month Appt Change In Weight (Kg)'] = output2['>=3 Month Weight'] - output2['1st Weight (Kg)']
output2.loc[(output2['1st Weight (Kg)'].isnull()) | (output2['>=3 Month Weight'].isnull()), \
            '1st - >=3 Month Appt Change In Weight (%)'] = None
output2.loc[(output2['1st Weight (Kg)'].notnull()) | (output2['>=3 Month Weight'].notnull()), \
            '1st - >=3 Month Appt Change In Weight (%)'] = output2['1st - >=3 Month Appt Change In Weight (Kg)']\
/ output2['1st Weight (Kg)']
output2.loc[(output2['1st Weight (Kg)'].isnull()) | (output2['Last  Weight (Kg)'].isnull()), \
            '1st - Last Appt Change in Weight (Kg)'] = None
output2.loc[(output2['1st Weight (Kg)'].notnull()) | (output2['Last  Weight (Kg)'].notnull()), \
            '1st - Last Appt Change in Weight (Kg)'] = output2['Last  Weight (Kg)'] - output2['1st Weight (Kg)']
output2.loc[(output2['1st Weight (Kg)'].isnull()) | (output2['Last  Weight (Kg)'].isnull()), \
            '1st - Last Appt Change in Weight (%)'] = None
output2.loc[(output2['1st Weight (Kg)'].notnull()) | (output2['Last  Weight (Kg)'].notnull()), \
            '1st - Last Appt Change in Weight (%)'] = output2['1st - Last Appt Change in Weight (Kg)']\
/ output2['1st Weight (Kg)']

personalnweight = personalnweightcount.merge(output2, how = 'outer', on = 'InternalID') #Merge it with main dataframe

#Format date as per templates
personalnweight['1st Appt Date'] = personalnweight['1st Appt Date'].dt.strftime('%d-%m-%Y')
personalnweight['>=3 Month Appt Date'] = personalnweight['>=3 Month Appt Date'].dt.strftime('%d-%m-%Y')
personalnweight['Last Appt Date'] = personalnweight['Last Appt Date'].dt.strftime('%d-%m-%Y')

personalnweight['Weight Notes'] = None #Create a notes column

# HBA1C

In [6]:
hbac = data.dropna(subset = ['Results Date']) #Drop rows where result date is blank
hbac = hbac.drop_duplicates(subset=['InternalID', 'Results Date'] , keep="last") 
#Drop rows where date is repeated for patient

res_count = hbac[['InternalID', 'Results Date']] #Select Relevant columns
res_count = res_count.groupby(['InternalID']).count() #Group to get no of result dates per patient
res_count = res_count.reset_index() #Get access to all columns
res_count.rename(columns = {'Results Date': '# of Results'}, inplace = True) #Rename column as per template

aboveallnresultcount = personalnweight.merge(res_count, how = 'outer', on = 'InternalID') #Merge it with main dataframe

temp1 = hbac.groupby(['InternalID']) #Group data on patients to get the earliest and latest date
temp1 = temp1.agg(Minimum_Date=('Results Date', np.min), Maximum_Date=('Results Date', np.max)) 
temp1 = temp1.reset_index() #Reset Index to access all columns
temp1['months_to_add'] = 90 #Get the date 3 months after the first visit
temp1['3 months'] = temp1['Minimum_Date'] + pd.TimedeltaIndex(temp1['months_to_add'], unit='D') 

three_month_list = temp1['3 months'].to_list() #Store the above result in a list
patient_list = temp1['InternalID'].to_list() #Store all patient ID in a list
three_month_exact_date = [] #Create an empty list
for (i,j) in zip(patient_list, three_month_list): #Loop through all the patient to get their 1st visit after 90 days
    temp_hbac = hbac[hbac['InternalID'] == i]
    date_working = temp_hbac[(temp_hbac['Results Date'] >= j)]
    date = date_working['Results Date'].min()
    three_month_exact_date.append(date)
temp1['>=3 Month Hba1c Date'] = three_month_exact_date #Create a column to get the 1st visit of each patient after 90 days

hbac = hbac[['InternalID', 'Results Date', 'HbA1c']] #Select relevant columns from a detail database

#Merge the detail database to get the HBA1C data for 1st visit, first visit after 3 months and latest visit of each patient
#and classify as Diabetic, Pre Diabetic and Non Diabetic as per result
output = temp1.merge(hbac, how='left', left_on=["InternalID", "Minimum_Date"], right_on=["InternalID","Results Date"])
output.rename(columns = {'Minimum_Date':'First Result Date','HbA1c':'First HbA1c Value'}, inplace = True)
output.loc[(output['First HbA1c Value']<= 5.6), 'First HbA1c Category'] = 'Non-Diabetic'
output.loc[(output['First HbA1c Value']>= 5.7) & (output['First HbA1c Value']<= 6.4), 'First HbA1c Category'] =\
'Pre-Diabetic'
output.loc[(output['First HbA1c Value']>= 6.5), 'First HbA1c Category'] = 'Diabetic'
output = output.merge(hbac, how='left', left_on=["InternalID", ">=3 Month Hba1c Date"], right_on=["InternalID", \
                                                                                                   "Results Date"])
output.rename(columns = {'HbA1c':'>=3 Month Hba1c'}, inplace = True)
output.loc[(output['>=3 Month Hba1c']<= 5.6), '>=3 Month Hba1c Category'] = 'Non-Diabetic'
output.loc[(output['>=3 Month Hba1c']>= 5.7) & (output['>=3 Month Hba1c']<= 6.4), '>=3 Month Hba1c Category'] =\
'Pre-Diabetic'
output.loc[(output['>=3 Month Hba1c']>= 6.5), '>=3 Month Hba1c Category'] = 'Diabetic'
output = output.merge(hbac, how='left', left_on=["InternalID", "Maximum_Date"], right_on=["InternalID","Results Date"])
output.rename(columns = {'Maximum_Date':'Last Hba1c Date','HbA1c':'Last Hba1c'}, inplace = True)
output.loc[(output['Last Hba1c']<= 5.6), 'Last HbA1c Category'] = 'Non-Diabetic'
output.loc[(output['Last Hba1c']>= 5.7) & (output['Last Hba1c']<= 6.4), 'Last HbA1c Category'] =\
'Pre-Diabetic'
output.loc[(output['Last Hba1c']>= 6.5), 'Last HbA1c Category'] = 'Diabetic'

output = output[['InternalID', 'First Result Date','First HbA1c Value', 'First HbA1c Category', '>=3 Month Hba1c Date',\
 '>=3 Month Hba1c', '>=3 Month Hba1c Category', 'Last Hba1c Date', 'Last Hba1c', 'Last HbA1c Category']]
#Select relevant columns

#Get absolute and % change of HBA1C between first & first report after 3 months and first report and last report
output.loc[(output['First HbA1c Value'].isnull()) | (output['>=3 Month Hba1c'].isnull()), \
            '1st - >3 Month Change Hba1c (#)'] = None
output.loc[(output['First HbA1c Value'].notnull()) | (output['>=3 Month Hba1c'].notnull()), \
            '1st - >3 Month Change Hba1c (#)'] = output['>=3 Month Hba1c'] - output['First HbA1c Value']
output.loc[(output['First HbA1c Value'].isnull()) | (output['>=3 Month Hba1c'].isnull()), \
            '1st - >3 Month Change Hba1c (%)'] = None
output.loc[(output['First HbA1c Value'].notnull()) | (output['>=3 Month Hba1c'].notnull()), \
            '1st - >3 Month Change Hba1c (%)'] = output['1st - >3 Month Change Hba1c (#)']\
/ output['First HbA1c Value']
output.loc[(output['First HbA1c Value'].isnull()) | (output['Last Hba1c'].isnull()), \
            '1st - Last  Change in Hba1c (#)'] = None
output.loc[(output['First HbA1c Value'].notnull()) | (output['Last Hba1c'].notnull()), \
            '1st - Last  Change in Hba1c (#)'] = output['Last Hba1c'] - output['First HbA1c Value']
output.loc[(output['First HbA1c Value'].isnull()) | (output['Last Hba1c'].isnull()), \
            '1st - Last  Change in Hba1c (%)'] = None
output.loc[(output['First HbA1c Value'].notnull()) | (output['Last Hba1c'].notnull()), \
            '1st - Last  Change in Hba1c (%)'] = output['1st - Last  Change in Hba1c (#)']\
/ output['First HbA1c Value']

allexceptdose = aboveallnresultcount.merge(output, how = 'outer', on = 'InternalID') 
#Merge with main dataframe

#Convert date as per required format
allexceptdose['First Result Date'] = allexceptdose['First Result Date'].dt.strftime('%d-%m-%Y')
allexceptdose['>=3 Month Hba1c Date'] = allexceptdose['>=3 Month Hba1c Date'].dt.strftime('%d-%m-%Y')
allexceptdose['Last Hba1c Date'] = allexceptdose['Last Hba1c Date'].dt.strftime('%d-%m-%Y')

allexceptdose['HBA1C Notes'] = None #Create a column for notes

# Dose Change

In [7]:
drug_count = data.dropna(subset = ['Drug name']) #Drop rows with empty drug name
drug_count = drug_count[['InternalID', 'Drug name']] # #Select relevant columns
drug_count = drug_count.groupby(['InternalID']).count() #Get number of drug per patient
drug_count = drug_count.reset_index() #Reset index to get access to all columns
drug_count.rename(columns = {'Drug name': '# of Drugs'}, inplace = True) #Rename column as per template

bp_dose = data[data['Drug category'] == 'Blood pressure'] #Filter data on Blood pressure
bp_dose = bp_dose[['InternalID','Dose change type']] #Select Relevant columns
bp_dose = bp_dose.dropna(subset = ['Dose change type']) #Drop rows with no data

bp_dose = pd.get_dummies(bp_dose) #Transform 1 column to 5 based on dosage change type

bp_dose.rename(columns = {'Dose change type_Ceased':'# Blood Pressure Drugs Ceased', \
                              'Dose change type_Decreased': '# Blood Pressure Drugs Decreased',\
                              'Dose change type_Increased\xa0': '# Blood Pressure Drugs Increased', \
                              'Dose change type_Unchanged': '# Blood Pressure Drugs Unchanged', \
                        'Dose change type_Started': '# Blood Pressure Drugs Started'}, inplace = True) #Rename the columns

bp_dose = bp_dose.groupby(['InternalID']).sum() #Group it with patient
bp_dose = bp_dose.reset_index() #Reset index to get access to all columns

bp_dose = bp_dose [['InternalID', '# Blood Pressure Drugs Ceased', '# Blood Pressure Drugs Decreased', \
             '# Blood Pressure Drugs Increased','# Blood Pressure Drugs Unchanged', '# Blood Pressure Drugs Started']]
#Rearrange column as per template

bp_dose = drug_count.merge(bp_dose, how = 'outer', on = 'InternalID') #Merge drug count data

non_bp_dose = data[data['Drug category'] != 'Blood pressure'] #Select data whose category is not blood pressure
non_bp_dose = non_bp_dose[['InternalID','Dose change type']] #Select Relevant columns
non_bp_dose = non_bp_dose.dropna(subset = ['Dose change type']) #Drop rows with no data

non_bp_dose = pd.get_dummies(non_bp_dose) #Transform 1 column to 5 based on dosage change type

non_bp_dose = pd.get_dummies(non_bp_dose) #Transform 1 column to 5 based on dosage change type
non_bp_dose.rename(columns = {'Dose change type_Ceased':'# Other Drugs Ceased', \
                              'Dose change type_Decreased': '# Other Drugs Decreased',\
                              'Dose change type_Increased\xa0': '# Other Drugs Increased', \
                              'Dose change type_Unchanged': '# Other Drugs Unchanged', \
                        'Dose change type_Started': '# Other Drugs Started'}, inplace = True) #Rename the columns

non_bp_dose = non_bp_dose.groupby(['InternalID']).sum() #Group it with patient
non_bp_dose = non_bp_dose.reset_index() #Reset index to get access to all columns

non_bp_dose = non_bp_dose [['InternalID', '# Other Drugs Ceased', '# Other Drugs Decreased', \
             '# Other Drugs Increased','# Other Drugs Unchanged', '# Other Drugs Started']] 
#Rearrange columns as per template

dose_change = bp_dose.merge(non_bp_dose, how = 'outer', on = 'InternalID') #Merge with above data in the same category

#Get total in drug change category
dose_change['# Drugs Ceased'] = dose_change['# Blood Pressure Drugs Ceased'] + dose_change['# Other Drugs Ceased']
dose_change['# Drugs Decreased'] = dose_change['# Blood Pressure Drugs Decreased'] + dose_change['# Other Drugs Decreased']
dose_change['# Drugs Increased'] = dose_change['# Blood Pressure Drugs Increased'] + dose_change['# Other Drugs Increased']
dose_change['#  Drugs Unchanged'] = dose_change['# Blood Pressure Drugs Unchanged'] + dose_change['# Other Drugs Unchanged']
dose_change['# Drugs Started'] = dose_change['# Blood Pressure Drugs Started'] + dose_change['# Other Drugs Started']

combined = allexceptdose.merge(dose_change, how = 'outer', on = 'InternalID') #Join it with main table

# Save output as excel

In [8]:
combined.to_excel('Result.xlsx', index = False) #Save as excel